In [1]:
#!pip install Census
#!pip install us

# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import us
from census import Census
from us import states
#from census_data import 

# Census API Key
from config import api_key
api_key
c = Census(api_key, year=2013)
c1 = Census(api_key,year=2017)

In [2]:
male_data=c1.acs1.get('B24010_019E', {'for': 'state:*'})
m_pd = pd.DataFrame(male_data)
m_pd = m_pd.rename(columns={"B24010_019E": "Male", 
                                       "state": "State Code"})
m_pd = m_pd[["State Code","Male"]]
m_pd.head()

,State Code,Male
0,01,72636.0
1,02,13748.0
2,04,130196.0
3,05,33942.0
4,06,922373.0


In [3]:
female_data=c1.acs1.get('B24010_155E', {'for': 'state:*'})
f_pd = pd.DataFrame(female_data)
f_pd = f_pd.rename(columns={"B24010_155E": "Female", 
                                       "state": "State Code"})
f_pd = f_pd[["State Code","Female"]]
f_pd.head()

,State Code,Female
0,01,123251.0
1,02,21991.0
2,04,221393.0
3,05,72013.0
4,06,1348070.0


In [75]:
# get state names
state_dict = us.states.mapping('fips', 'name')
# Append DataFrame
mf_df = pd.merge(f_pd, m_pd, on='State Code', how='inner')
mf_df=mf_df.fillna(0)

mf_df["Total"] = mf_df["Male"].astype(int) + mf_df["Female"].astype(int)
mf_df["State"] = ''
mf_df["Type"] = ''
#mf_df["State"] = state_dict[mf_df["State Code"]] 
state_dict = us.states.mapping('fips', 'name')
for index, row in mf_df.iterrows():
    mf_df.loc[index,'State'] = state_dict[mf_df.loc[index,'State Code']]
    mf_df.loc[index,'Type'] = 'IT'


mf_df = mf_df[["State Code", "State", "Type","Male", "Female","Total"]]
mf_df.sort_values('Total', ascending=False).head()

,State Code,State,Type,Male,Female,Total
4,06,California,IT,922373.0,1348070.0,2270443
43,48,Texas,IT,573538.0,894007.0,1467545
32,36,New York,IT,331486.0,673544.0,1005030
9,12,Florida,IT,299000.0,629087.0,928087
13,17,Illinois,IT,252640.0,442071.0,694711


In [5]:
#used functions
def func_merge_df(df1, df2, column):
    mf_df = pd.merge(df1, df2, on=column, how='inner')
    return mf_df

def func_getdata(key1, key2, jobtype):
    male_data= []
    female_data = []
    state_dict = us.states.mapping('fips', 'name')
    
    male_data=c1.acs1.get(key1, {'for': 'state:*'})
    female_data=c1.acs1.get(key2, {'for': 'state:*'})
    m_pd = pd.DataFrame(male_data)
    f_pd = pd.DataFrame(female_data)
    # Merge the dataframe
    mf_df = func_merge_df(df1=f_pd, df2=m_pd, column='state')
    mf_df=mf_df.fillna(0)
    # Get the total 
    mf_df["State Name"] = ''
    mf_df["Type"] = ''
    # Get the State name
    for index, row in mf_df.iterrows():
        mf_df.loc[index,'State Name'] = state_dict[mf_df.loc[index,'state']]
        mf_df.loc[index,'Type'] = jobtype
    return mf_df

In [ ]:
#main program
cnt = 1
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_155E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)



for index, row in input_df.iterrows(): 
    filter_df = input_df[input_df['jobtype']==input_df.loc[index,'jobtype']]
    key1_val = ','.join(str(v) for v in filter_df['Male'])
    key2_val = ','.join(str(v) for v in filter_df['Female'])
    jobtypeVal = ','.join(str(v) for v in filter_df['jobtype'])
 
    rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
    rs_data
    rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
    rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
    rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
    rs_data.sort_values('Total', ascending=False).head() 

# First time for the header
    if cnt == 1:
        rs_data.to_csv("job_census_data.csv", encoding="utf-8", index=False)
        cnt = cnt + 1

    if cnt > 1:
        with open('job_census_data.csv', 'a') as f:
            rs_data.to_csv(f, header=False, index=False)

print("Job Completed")

In [6]:
cnt = 1
input_data=[{'Male':'B24040_007E','Female':'B24040_111E','jobtype':'Manufacturing'},
           {'Male':'B24010_019E','Female':'B24010_155E','jobtype':'IT'},
           {'Male':'B24010_037E','Female':'B24010_188E','jobtype':'Social Workers'},
           {'Male':'B24010_017E','Female':'B24010_168E','jobtype':'Finance'}
          ]
input_df = pd.DataFrame(input_data)
 
 
key1_val = 'B24040_007E'
key2_val = 'B24040_111E'
jobtypeVal = 'Manufacturing'

rs_data = func_getdata(key1 = key1_val, key2 = key2_val, jobtype = jobtypeVal)
rs_data
rs_data = rs_data.rename(columns={key1_val: "Male",key2_val:"Female","state": "State Code"})
rs_data["Total"] = rs_data["Male"].astype(int) + rs_data["Female"].astype(int)
rs_data = rs_data[["State Code", "State Name", "Type","Male", "Female","Total"]]
rs_data.sort_values('Total', ascending=False).head() 

,State Code,State Name,Type,Male,Female,Total
4,06,California,Manufacturing,1020705.0,421201.0,1441906
43,48,Texas,Manufacturing,730346.0,242316.0,972662
22,26,Michigan,Manufacturing,570745.0,191694.0,762439
35,39,Ohio,Manufacturing,543443.0,187144.0,730587
13,17,Illinois,Manufacturing,463010.0,187851.0,650861


In [68]:
manufacturing_data = c.acs1.get(("NAME", "B24040_008E", "B24040_112M","B24040_009E", "B24040_113E" , "B24040_010E", "B24040_114E","B24040_014E", "B24040_118E", "B24040_043E", "B24040_120E"),{'for': 'state: *'})
manufacturing_data_df=pd.DataFrame(manufacturing_data)   
manufacturing_data_df = manufacturing_data_df.rename(columns={"B24040_008E": "Male-Food",
                                      "B24040_112M": "Female-Food",
                                    "B24040_009E" : "Male-Beverage and Tobacco",
                                    "B24040_113E" : "Female-Beverage and Tobacco",
                                    "B24040_010E" : "Male-Textile Mills and Products",
                                    "B24040_114E" : "Female-Textile Mills and Products",
                                    "B24040_014E" : "Male-Paper",
                                    "B24040_118E" : "Female-Paper",
                                    "B24040_043E" : "Male-Petroleum and Coal",
                                    "B24040_120E" : "Female-Petroleum and Coal"
                                                              })
manufacturing_data_df.set_index("NAME", inplace=True)
manufacturing_data_df = manufacturing_data_df[np.isfinite(manufacturing_data_df['Male-Food'])]
manufacturing_data_df["Total Food"] = manufacturing_data_df["Male-Food"].astype(int) + manufacturing_data_df["Female-Food"].astype(int)
manufacturing_data_df["Total Textile"] = manufacturing_data_df["Male-Textile Mills and Products"].astype(int) + manufacturing_data_df["Female-Textile Mills and Products"].astype(int)
manufacturing_data_df["Total Beverage and Tobacco"] = manufacturing_data_df["Male-Beverage and Tobacco"].astype(int) + manufacturing_data_df["Female-Beverage and Tobacco"].astype(int)
manufacturing_data_df["Total Paper"] = manufacturing_data_df["Male-Paper"].astype(int) + manufacturing_data_df["Female-Paper"].astype(int)
manufacturing_data_df["Total Petroleum and Coal"] = manufacturing_data_df["Male-Petroleum and Coal"].astype(int) + manufacturing_data_df["Female-Petroleum and Coal"].astype(int)
manufacturing_data_df.head()


,Male-Food,Male-Beverage and Tobacco,Male-Textile Mills and Products,Male-Paper,Male-Petroleum and Coal,Female-Food,Female-Beverage and Tobacco,Female-Textile Mills and Products,Female-Paper,Female-Petroleum and Coal,state,Total Food,Total Textile,Total Beverage and Tobacco,Total Paper,Total Petroleum and Coal
NAME,,,,,,,,,,,,,,,,
Alabama,14531.0,824.0,5561.0,9459.0,1546.0,1715.0,284.0,3313.0,2198.0,336.0,01,16246,8874,1108,11657,1882
Alaska,3840.0,175.0,0.0,0.0,561.0,340.0,76.0,0.0,0.0,150.0,02,4180,0,251,0,711
Arizona,5213.0,1976.0,695.0,2025.0,883.0,878.0,314.0,391.0,1230.0,0.0,04,6091,1086,2290,3255,883
Arkansas,23662.0,422.0,385.0,5430.0,786.0,1803.0,146.0,166.0,1857.0,406.0,05,25465,551,568,7287,1192
California,87129.0,28369.0,8080.0,12760.0,6007.0,4475.0,9777.0,7265.0,3467.0,3914.0,06,91604,15345,38146,16227,9921


In [30]:
food_df=manufacturing_data_df[["Male-Food", "Female-Food"]]
food_df["Total Food"] = food_df["Male-Food"].astype(int) + food_df["Female-Food"].astype(int)
food_df
food_df.sort_values('Total Food', ascending=False).head()

/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Food,Female-Food,Total Food
NAME,,,
California,87129.0,4475.0,91604
Illinois,51501.0,2673.0,54174
Texas,47602.0,3310.0,50912
Pennsylvania,39851.0,2004.0,41855
Wisconsin,39644.0,2055.0,41699


In [34]:
textile_df=manufacturing_data_df[["Male-Textile Mills and Products", "Female-Textile Mills and Products"]]
textile_df["Total Textile"] = textile_df["Male-Textile Mills and Products"].astype(int) + textile_df["Female-Textile Mills and Products"].astype(int)
textile_df
textile_df.sort_values('Total Textile', ascending=False).head()

/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Textile Mills and Products,Female-Textile Mills and Products,Total Textile
NAME,,,
Georgia,26916.0,15491.0,42407
North Carolina,16962.0,12995.0,29957
South Carolina,12030.0,6497.0,18527
California,8080.0,7265.0,15345
Alabama,5561.0,3313.0,8874


In [36]:
tobacco_df=manufacturing_data_df[["Male-Beverage and Tobacco", "Female-Beverage and Tobacco"]]
tobacco_df["Total Beverage and Tobacco"] = tobacco_df["Male-Beverage and Tobacco"].astype(int) + tobacco_df["Female-Beverage and Tobacco"].astype(int)
tobacco_df
tobacco_df.sort_values('Total Beverage and Tobacco', ascending=False).head()


/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Beverage and Tobacco,Female-Beverage and Tobacco,Total Beverage and Tobacco
NAME,,,
California,28369.0,9777.0,38146
Texas,11818.0,3501.0,15319
North Carolina,8423.0,3307.0,11730
Georgia,6413.0,3989.0,10402
New York,7516.0,2566.0,10082


In [37]:
paper_df=manufacturing_data_df[["Male-Paper", "Female-Paper"]]
paper_df["Total Paper"] = paper_df["Male-Paper"].astype(int) + paper_df["Female-Paper"].astype(int)
paper_df
paper_df.sort_values('Total Paper', ascending=False).head()

/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Paper,Female-Paper,Total Paper
NAME,,,
Wisconsin,23145.0,9041.0,32186
Pennsylvania,16930.0,5719.0,22649
Ohio,15551.0,4927.0,20478
Georgia,16135.0,3462.0,19597
Texas,11535.0,4877.0,16412


In [39]:
coal_df=manufacturing_data_df[["Male-Petroleum and Coal", "Female-Petroleum and Coal"]]
coal_df["Total Petroleum and Coal"] = coal_df["Male-Petroleum and Coal"].astype(int) + coal_df["Female-Petroleum and Coal"].astype(int)
coal_df
coal_df.sort_values('Total Petroleum and Coal', ascending=False).head()


/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Petroleum and Coal,Female-Petroleum and Coal,Total Petroleum and Coal
NAME,,,
Texas,19775.0,15064.0,34839
California,6007.0,3914.0,9921
Louisiana,4330.0,1754.0,6084
Pennsylvania,3231.0,818.0,4049
Oklahoma,2300.0,1565.0,3865


In [73]:
manufacturing_data = c.acs1.get(("NAME", "B24040_008E", "B24040_112M","B24040_009E", "B24040_113E" , "B24040_010E", "B24040_114E","B24040_014E", "B24040_118E", "B24040_043E", "B24040_120E"),{'for': 'state: 06, 48, 26, 39, 17'})
manufacturing_data_df=pd.DataFrame(manufacturing_data)   
manufacturing_data_df = manufacturing_data_df.rename(columns={"B24040_008E": "Male-Food",
                                      "B24040_112M": "Female-Food",
                                    "B24040_009E" : "Male-Beverage and Tobacco",
                                    "B24040_113E" : "Female-Beverage and Tobacco",
                                    "B24040_010E" : "Male-Textile Mills and Products",
                                    "B24040_114E" : "Female-Textile Mills and Products",
                                    "B24040_014E" : "Male-Paper",
                                    "B24040_118E" : "Female-Paper",
                                    "B24040_043E" : "Male-Petroleum and Coal",
                                    "B24040_120E" : "Female-Petroleum and Coal"
                                                              })
manufacturing_data_df.set_index("NAME", inplace=True)
manufacturing_data_df = manufacturing_data_df[np.isfinite(manufacturing_data_df['Male-Food'])]
manufacturing_data_df["Total Food"] = manufacturing_data_df["Male-Food"].astype(int) + manufacturing_data_df["Female-Food"].astype(int)
manufacturing_data_df["Total Textile"] = manufacturing_data_df["Male-Textile Mills and Products"].astype(int) + manufacturing_data_df["Female-Textile Mills and Products"].astype(int)
manufacturing_data_df["Total Beverage and Tobacco"] = manufacturing_data_df["Male-Beverage and Tobacco"].astype(int) + manufacturing_data_df["Female-Beverage and Tobacco"].astype(int)
manufacturing_data_df["Total Paper"] = manufacturing_data_df["Male-Paper"].astype(int) + manufacturing_data_df["Female-Paper"].astype(int)
manufacturing_data_df["Total Petroleum and Coal"] = manufacturing_data_df["Male-Petroleum and Coal"].astype(int) + manufacturing_data_df["Female-Petroleum and Coal"].astype(int)
manufacturing_data_df.head()

,Male-Food,Male-Beverage and Tobacco,Male-Textile Mills and Products,Male-Paper,Male-Petroleum and Coal,Female-Food,Female-Beverage and Tobacco,Female-Textile Mills and Products,Female-Paper,Female-Petroleum and Coal,state,Total Food,Total Textile,Total Beverage and Tobacco,Total Paper,Total Petroleum and Coal
NAME,,,,,,,,,,,,,,,,
California,87129.0,28369.0,8080.0,12760.0,6007.0,4475.0,9777.0,7265.0,3467.0,3914.0,06,91604,15345,38146,16227,9921
Illinois,51501.0,5600.0,1581.0,9785.0,1820.0,2673.0,1975.0,2254.0,4063.0,955.0,17,54174,3835,7575,13848,2775
Michigan,20080.0,4139.0,1659.0,9452.0,1520.0,1442.0,539.0,1561.0,1972.0,314.0,26,21522,3220,4678,11424,1834
Ohio,32157.0,4378.0,3490.0,15551.0,1965.0,1877.0,808.0,1313.0,4927.0,1642.0,39,34034,4803,5186,20478,3607
Texas,47602.0,11818.0,4013.0,11535.0,19775.0,3310.0,3501.0,2909.0,4877.0,15064.0,48,50912,6922,15319,16412,34839


In [74]:
top_5_total=manufacturing_data_df[["Total Food", "Total Textile", "Total Beverage and Tobacco", "Total Paper", "Total Petroleum and Coal"]]
top_5_total


,Total Food,Total Textile,Total Beverage and Tobacco,Total Paper,Total Petroleum and Coal
NAME,,,,,
California,91604,15345,38146,16227,9921
Illinois,54174,3835,7575,13848,2775
Michigan,21522,3220,4678,11424,1834
Ohio,34034,4803,5186,20478,3607
Texas,50912,6922,15319,16412,34839


In [72]:
IT_data = c.acs1.get(("NAME","B24010_020E", "B24010_171E", "B24010_022E",  "B24010_173E", "B24010_023E", "B24010_174E", "B24010_024E",  "B24010_175E" ),{'for': 'state:*'})
IT_data_df=pd.DataFrame(IT_data)   
IT_data_df = IT_data_df.rename(columns={"B24010_020E": "Male-Computer and Math",
                                      "B24010_171E": "Female-Computer and Math",
                                    "B24010_022E" : "Male-Information Research Scientists and Analysts",
                                    "B24010_173E" : "Female-Information Research Scientists and Analysts",
                                    "B24010_023E" : "Male-Software developers and programmers",
                                    "B24010_174E" : "Female-Software developers and programmers",
                                    "B24010_024E" : "Male-Database and systems administrators and network architects",
                                    "B24010_175E" : "Female-Database and systems administrators and network architects",
                                    
                                                              })
IT_data_df.set_index("NAME", inplace=True)

IT_data_df.head()

,Male-Computer and Math,Male-Information Research Scientists and Analysts,Male-Software developers and programmers,Male-Database and systems administrators and network architects,Female-Computer and Math,Female-Information Research Scientists and Analysts,Female-Software developers and programmers,Female-Database and systems administrators and network architects,state
NAME,,,,,,,,,
Alabama,27111.0,2762.0,12323.0,4430.0,10731.0,2375.0,2768.0,1715.0,01
Alaska,4766.0,652.0,450.0,805.0,1169.0,230.0,467.0,185.0,02
Arizona,60857.0,7206.0,26562.0,7458.0,19254.0,4306.0,7028.0,1880.0,04
Arkansas,15654.0,1896.0,6546.0,2224.0,5748.0,1447.0,1077.0,639.0,05
California,391822.0,45021.0,219231.0,38662.0,127253.0,24802.0,59782.0,10082.0,06


In [46]:
math_df=IT_data_df[["Male-Computer and Math", "Female-Computer and Math"]]
math_df["Total Computer and Math"] = math_df["Male-Computer and Math"].astype(int) + math_df["Female-Computer and Math"].astype(int)
math_df
math_df.sort_values('Total Computer and Math', ascending=False).head()
    

/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Computer and Math,Female-Computer and Math,Total Computer and Math
NAME,,,
California,391822.0,127253.0,519075
Texas,230197.0,77666.0,307863
New York,158005.0,54977.0,212982
Virginia,140709.0,49930.0,190639
Florida,131718.0,43618.0,175336


In [47]:
research_df=IT_data_df[["Male-Information Research Scientists and Analysts", "Female-Information Research Scientists and Analysts"]]
research_df["Total Information Research Scientists and Analysts"] = research_df["Male-Information Research Scientists and Analysts"].astype(int) + research_df["Female-Information Research Scientists and Analysts"].astype(int)
research_df
research_df.sort_values('Total Information Research Scientists and Analysts', ascending=False).head()


/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Information Research Scientists and Analysts,Female-Information Research Scientists and Analysts,Total Information Research Scientists and Analysts
NAME,,,
California,45021.0,24802.0,69823
Texas,33065.0,16540.0,49605
Virginia,25791.0,11890.0,37681
New York,22702.0,10912.0,33614
Illinois,19416.0,11227.0,30643


In [48]:
software_df=IT_data_df[["Male-Software developers and programmers", "Female-Software developers and programmers"]]
software_df["Total Software developers and programmers"] = software_df["Male-Software developers and programmers"].astype(int) + software_df["Female-Software developers and programmers"].astype(int)
software_df
software_df.sort_values('Total Software developers and programmers', ascending=False).head()


/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Software developers and programmers,Female-Software developers and programmers,Total Software developers and programmers
NAME,,,
California,219231.0,59782.0,279013
Texas,98218.0,27220.0,125438
New York,68385.0,21434.0,89819
Washington,64944.0,13402.0,78346
Florida,56889.0,13035.0,69924


In [49]:
database_df=IT_data_df[["Male-Database and systems administrators and network architects", "Female-Database and systems administrators and network architects"]]
database_df["Total Database and systems administrators and network architects"] = database_df["Male-Database and systems administrators and network architects"].astype(int) + database_df["Female-Database and systems administrators and network architects"].astype(int)
database_df
database_df.sort_values('Total Database and systems administrators and network architects', ascending=False).head()


/Users/sofiecoopersmith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Male-Database and systems administrators and network architects,Female-Database and systems administrators and network architects,Total Database and systems administrators and network architects
NAME,,,
California,38662.0,10082.0,48744
Texas,27047.0,7956.0,35003
New York,19341.0,4942.0,24283
Virginia,17499.0,5255.0,22754
Florida,15342.0,4589.0,19931


In [80]:
IT_data = c.acs1.get(("NAME","B24010_020E", "B24010_171E", "B24010_022E",  "B24010_173E", "B24010_023E", "B24010_174E", "B24010_024E",  "B24010_175E" ),{'for': 'state:06, 48,36,12,17'})
IT_data_df=pd.DataFrame(IT_data)   
IT_data_df = IT_data_df.rename(columns={"B24010_020E": "Male-Computer and Math",
                                      "B24010_171E": "Female-Computer and Math",
                                    "B24010_022E" : "Male-Information Research Scientists and Analysts",
                                    "B24010_173E" : "Female-Information Research Scientists and Analysts",
                                    "B24010_023E" : "Male-Software developers and programmers",
                                    "B24010_174E" : "Female-Software developers and programmers",
                                    "B24010_024E" : "Male-Database and systems administrators and network architects",
                                    "B24010_175E" : "Female-Database and systems administrators and network architects",
                                    
                                                              })

IT_data_df["Total Computer and Math"] = IT_data_df["Male-Computer and Math"].astype(int) + IT_data_df["Female-Computer and Math"].astype(int)
IT_data_df["Total Information Research Scientists and Analysts"] = IT_data_df["Male-Information Research Scientists and Analysts"].astype(int) + IT_data_df["Female-Information Research Scientists and Analysts"].astype(int)
IT_data_df["Total Software developers and programmers"] = IT_data_df["Male-Software developers and programmers"].astype(int) + IT_data_df["Female-Software developers and programmers"].astype(int)
IT_data_df["Total Database and systems administrators and network architects"] = IT_data_df["Male-Database and systems administrators and network architects"].astype(int) + IT_data_df["Female-Database and systems administrators and network architects"].astype(int)
IT_data_df.head()
IT_data_df.set_index("NAME", inplace=True)

IT_data_df.head()

,Male-Computer and Math,Male-Information Research Scientists and Analysts,Male-Software developers and programmers,Male-Database and systems administrators and network architects,Female-Computer and Math,Female-Information Research Scientists and Analysts,Female-Software developers and programmers,Female-Database and systems administrators and network architects,state,Total Computer and Math,Total Information Research Scientists and Analysts,Total Software developers and programmers,Total Database and systems administrators and network architects
NAME,,,,,,,,,,,,,
California,391822.0,45021.0,219231.0,38662.0,127253.0,24802.0,59782.0,10082.0,06,519075,69823,279013,48744
Florida,131718.0,17931.0,56889.0,15342.0,43618.0,8952.0,13035.0,4589.0,12,175336,26883,69924,19931
Illinois,123669.0,19416.0,50302.0,15431.0,44869.0,11227.0,15274.0,3474.0,17,168538,30643,65576,18905
New York,158005.0,22702.0,68385.0,19341.0,54977.0,10912.0,21434.0,4942.0,36,212982,33614,89819,24283
Texas,230197.0,33065.0,98218.0,27047.0,77666.0,16540.0,27220.0,7956.0,48,307863,49605,125438,35003


In [82]:
top_5_total_IT=IT_data_df[["Total Computer and Math", "Total Information Research Scientists and Analysts", "Total Software developers and programmers", "Total Database and systems administrators and network architects"]]
top_5_total_IT

,Total Computer and Math,Total Information Research Scientists and Analysts,Total Software developers and programmers,Total Database and systems administrators and network architects
NAME,,,,
California,519075,69823,279013,48744
Florida,175336,26883,69924,19931
Illinois,168538,30643,65576,18905
New York,212982,33614,89819,24283
Texas,307863,49605,125438,35003
